In [1]:
!pip install SPARQLWrapper networkx pandas tqdm rdflib requests retrying python-dateutil

In [2]:
"""
Sistema de Enriquecimiento de Datos Médicos desde DBpedia y Wikidata
=====================================================================
Este módulo consulta DBpedia y Wikidata para obtener información médica estructurada
que enriquecerá el corpus CoWeSe para el sistema RAG.
"""

from SPARQLWrapper import SPARQLWrapper, JSON
from retrying import retry
import time
import pandas as pd
import json
from typing import List, Dict, Any
from tqdm import tqdm

# Endpoints SPARQL
DBPEDIA_SPARQL = "https://dbpedia.org/sparql"
DBPEDIA_ES_SPARQL = "https://es.dbpedia.org/sparql"
WIKIDATA_SPARQL = "https://query.wikidata.org/sparql"

# User agent recomendado
USER_AGENT = "MedicalKG-RAG/1.0 (educational project; e.santofimio@uniandes.edu.co)"

print("Módulo de enriquecimiento médico inicializado")
print(f"DBpedia EN: {DBPEDIA_SPARQL}")
print(f"DBpedia ES: {DBPEDIA_ES_SPARQL}")
print(f"Wikidata: {WIKIDATA_SPARQL}")

Módulo de enriquecimiento médico inicializado
DBpedia EN: https://dbpedia.org/sparql
DBpedia ES: https://es.dbpedia.org/sparql
Wikidata: https://query.wikidata.org/sparql


In [3]:
@retry(wait_exponential_multiplier=1000, wait_exponential_max=10000, stop_max_attempt_number=5)
def run_sparql_query(query: str, endpoint: str = DBPEDIA_SPARQL, timeout: int = 60) -> Dict:
    """
    Ejecutar consulta SPARQL con reintentos automáticos
    
    Args:
        query: Consulta SPARQL
        endpoint: Endpoint SPARQL a usar
        timeout: Timeout en segundos
        
    Returns:
        Resultados en formato JSON
    """
    sparql = SPARQLWrapper(endpoint, agent=USER_AGENT)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    sparql.setTimeout(timeout)
    
    try:
        result = sparql.query().convert()
        return result
    except Exception as e:
        print(f"Error en consulta SPARQL: {e}")
        raise


In [6]:
def get_diseases_from_dbpedia(limit: int = 100) -> pd.DataFrame:
    """
    Obtener enfermedades desde DBpedia con descripciones y propiedades
    
    Args:
        limit: Número máximo de enfermedades a recuperar
        
    Returns:
        DataFrame con información de enfermedades
    """
    query = f"""
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbp: <http://dbpedia.org/property/>
    
    SELECT DISTINCT 
        ?disease 
        ?label 
        ?abstract
        (GROUP_CONCAT(DISTINCT ?symptom; separator="|") AS ?symptoms)
        (GROUP_CONCAT(DISTINCT ?treatment; separator="|") AS ?treatments)
        (GROUP_CONCAT(DISTINCT ?cause; separator="|") AS ?causes)
    WHERE {{
        # Enfermedades
        ?disease a dbo:Disease .
        ?disease rdfs:label ?label .
        FILTER(LANG(?label) = "es" || LANG(?label) = "en")
        
        # Resumen/descripción
        OPTIONAL {{ 
            ?disease dbo:abstract ?abstract .
            FILTER(LANG(?abstract) = "es")
        }}
        
        # Síntomas
        OPTIONAL {{ ?disease dbo:symptom ?symptom }}
        
        # Tratamientos
        OPTIONAL {{ ?disease dbo:treatment ?treatment }}
        
        # Causas
        OPTIONAL {{ ?disease dbp:causes ?cause }}
    }}
    GROUP BY ?disease ?label ?abstract
    LIMIT {limit}
    """
    
    print(f"Consultando DBpedia por {limit} enfermedades...")
    results = run_sparql_query(query, endpoint=DBPEDIA_SPARQL)
    
    # Convertir a DataFrame
    bindings = results.get('results', {}).get('bindings', [])
    
    data = []
    for binding in bindings:
        data.append({
            'uri': binding.get('disease', {}).get('value', ''),
            'nombre': binding.get('label', {}).get('value', ''),
            'descripcion': binding.get('abstract', {}).get('value', ''),
            'sintomas': binding.get('symptoms', {}).get('value', '').split('|') if binding.get('symptoms', {}).get('value') else [],
            'tratamientos': binding.get('treatments', {}).get('value', '').split('|') if binding.get('treatments', {}).get('value') else [],
            'causas': binding.get('causes', {}).get('value', '').split('|') if binding.get('causes', {}).get('value') else []
        })
    
    df = pd.DataFrame(data)
    print(f"Obtenidas {len(df)} enfermedades de DBpedia")
    return df

# Ejecutar consulta
diseases_df = get_diseases_from_dbpedia(limit=300)

Consultando DBpedia por 300 enfermedades...
Obtenidas 300 enfermedades de DBpedia


In [7]:
# Mostrar primeros resultados
print(f"Muestra de enfermedades obtenidas:")
print(f"Total: {len(diseases_df)} enfermedades")

if not diseases_df.empty:
    for idx, row in diseases_df.head(5).iterrows():
        print(f"{row['nombre']}")
        if row['descripcion']:
            desc = row['descripcion'][:200] + "..." if len(row['descripcion']) > 200 else row['descripcion']
            print(f" {desc}")
        if row['sintomas']:
            print(f"Síntomas: {', '.join([s.split('/')[-1] for s in row['sintomas'][:3]])}")
        if row['tratamientos']:
            print(f"Tratamientos: {', '.join([t.split('/')[-1] for t in row['tratamientos'][:3]])}")
        print()
else:
    print("No se obtuvieron resultados de DBpedia")

Muestra de enfermedades obtenidas:
Total: 300 enfermedades
Encefalitis japonesa
Síntomas: Seizure
Tratamientos: Symptomatic_treatment

Insomnio familiar letal
Síntomas: Ataxia, Diplopia
Tratamientos: Symptomatic_treatment

Fractura de pelvis
Tratamientos: Embolization, Fluid_replacement, Pelvic_binder

Sirenomelia
Tratamientos: Surgery

Jones fracture
Tratamientos: Orthopedic_cast



In [9]:
def get_diseases_spanish_dbpedia(limit: int = 50) -> pd.DataFrame:
    """
    Obtener enfermedades desde DBpedia en español con información rica
    
    Args:
        limit: Número de enfermedades
        
    Returns:
        DataFrame con información en español
    """
    query = f"""
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    
    SELECT DISTINCT ?disease ?label ?abstract
    WHERE {{
        ?disease a dbo:Disease .
        ?disease rdfs:label ?label .
        FILTER(LANG(?label) = "es")
        
        ?disease dbo:abstract ?abstract .
        FILTER(LANG(?abstract) = "es")
        
        # Solo enfermedades con resumen en español
        FILTER(STRLEN(?abstract) > 200)
    }}
    LIMIT {limit}
    """
    
    print(f"🔍 Consultando DBpedia ES por {limit} enfermedades...")
    try:
        results = run_sparql_query(query, endpoint=DBPEDIA_ES_SPARQL)
        
        bindings = results.get('results', {}).get('bindings', [])
        
        data = []
        for binding in bindings:
            data.append({
                'uri': binding.get('disease', {}).get('value', ''),
                'nombre': binding.get('label', {}).get('value', ''),
                'descripcion': binding.get('abstract', {}).get('value', '')
            })
        
        df = pd.DataFrame(data)
        print(f"✓ Obtenidas {len(df)} enfermedades de DBpedia ES")
        return df
    except Exception as e:
        print(f"Error consultando DBpedia ES: {e}")
        return pd.DataFrame()

# Ejecutar consulta
diseases_es_df = get_diseases_spanish_dbpedia(limit=200)

🔍 Consultando DBpedia ES por 200 enfermedades...
✓ Obtenidas 200 enfermedades de DBpedia ES


In [10]:
# Mostrar resultados en español
print(f"Enfermedades con descripciones en español:")

if not diseases_es_df.empty:
    print(f"Total: {len(diseases_es_df)} enfermedades")
    
    for idx, row in diseases_es_df.head(3).iterrows():
        print(f"{row['nombre']}")
        print(f"Descripción:")
        desc = row['descripcion']
        print(f"{desc}")
else:
    print("No se obtuvieron resultados de DBpedia ES")

Enfermedades con descripciones en español:
Total: 200 enfermedades
Arco aórtico doble
Descripción:
El arco aórtico doble es una malformación cardiovascular congénita relativamente rara. El AAD es una anomalía del arco aórtico en el que dos arcos aórticos forman un anillo vascular completo que puede comprimir la tráquea y / o el esófago . ​ ​ Por lo general, hay un arco derecho más grande (dominante) detrás y un arco aórtico izquierdo más pequeño ( hipoplásico ) delante de la tráquea / esófago. Los dos arcos se unen para formar la aorta descendente, que generalmente está en el lado izquierdo (pero puede ser del lado derecho o en la línea media). En algunos casos, el extremo del arco aórtico izquierdo más pequeño se cierra (arco atrésico izquierdo) y el tejido vascular se convierte en un cordón fibroso. Aunque en estos casos no está presente un anillo completo de dos arcos aórticos patentes, el término "anillo vascular" es el término genérico aceptado incluso en estas anomalías. Los sínt

In [11]:
def get_wikidata_diseases(limit: int = 500, offset: int = 0) -> pd.DataFrame:
    """
    Obtener enfermedades de Wikidata con relaciones estructuradas
    
    Args:
        limit: Límite de resultados
        offset: Desplazamiento para paginación
        
    Returns:
        DataFrame con enfermedades y relaciones
    """
    query = f"""
    SELECT DISTINCT 
        ?disease ?diseaseLabel 
        ?symptomLabel 
        ?treatmentLabel
        ?causeLabel
        ?drugLabel
    WHERE {{
        # Enfermedades (instancia o subclase de enfermedad)
        ?disease wdt:P31/wdt:P279* wd:Q12136 .
        
        # Síntomas
        OPTIONAL {{ 
            ?disease wdt:P780 ?symptom .
        }}
        
        # Tratamientos
        OPTIONAL {{ 
            ?disease wdt:P2176 ?treatment .
        }}
        
        # Causas
        OPTIONAL {{ 
            ?disease wdt:P828 ?cause .
        }}
        
        # Medicamentos usados
        OPTIONAL {{
            ?disease wdt:P2176 ?drug .
            ?drug wdt:P31/wdt:P279* wd:Q12140 .
        }}
        
        # Etiquetas en español
        SERVICE wikibase:label {{ 
            bd:serviceParam wikibase:language "es,en" . 
        }}
    }}
    LIMIT {limit}
    OFFSET {offset}
    """
    
    print(f"Consultando Wikidata (offset={offset}, limit={limit})...")
    results = run_sparql_query(query, endpoint=WIKIDATA_SPARQL, timeout=120)
    
    bindings = results.get('results', {}).get('bindings', [])
    
    data = []
    for binding in bindings:
        data.append({
            'uri': binding.get('disease', {}).get('value', ''),
            'enfermedad': binding.get('diseaseLabel', {}).get('value', ''),
            'sintoma': binding.get('symptomLabel', {}).get('value', ''),
            'tratamiento': binding.get('treatmentLabel', {}).get('value', ''),
            'causa': binding.get('causeLabel', {}).get('value', ''),
            'medicamento': binding.get('drugLabel', {}).get('value', '')
        })
    
    df = pd.DataFrame(data)
    print(f"✓ Obtenidos {len(df)} registros de Wikidata")
    return df

# Ejecutar consulta con retry
try:
    wikidata_df = get_wikidata_diseases(limit=600)
except Exception as e:
    print(f"Error: {e}")
    print("Intentando con menos resultados...")
    wikidata_df = get_wikidata_diseases(limit=200)

Consultando Wikidata (offset=0, limit=600)...
✓ Obtenidos 600 registros de Wikidata


In [12]:
# Agrupar datos por enfermedad
print(f"Procesando datos de Wikidata...")

if not wikidata_df.empty:
    # Agrupar por enfermedad
    wikidata_grouped = wikidata_df.groupby('enfermedad').agg({
        'uri': 'first',
        'sintoma': lambda x: list(set([s for s in x if s])),
        'tratamiento': lambda x: list(set([t for t in x if t])),
        'causa': lambda x: list(set([c for c in x if c])),
        'medicamento': lambda x: list(set([m for m in x if m]))
    }).reset_index()
    
    print(f"Total de enfermedades únicas: {len(wikidata_grouped)}")
    
    # Mostrar ejemplos
    for idx, row in wikidata_grouped.head(5).iterrows():
        print(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        print(f"{row['enfermedad']}")
        
        if row['sintoma']:
            sintomas = [s for s in row['sintoma'] if s][:5]
            if sintomas:
                print(f"Síntomas: {', '.join(sintomas)}")
        
        if row['tratamiento']:
            tratamientos = [t for t in row['tratamiento'] if t][:3]
            if tratamientos:
                print(f"Tratamientos: {', '.join(tratamientos)}")
        
        if row['medicamento']:
            medicamentos = [m for m in row['medicamento'] if m][:3]
            if medicamentos:
                print(f" Medicamentos: {', '.join(medicamentos)}")
        
        if row['causa']:
            causas = [c for c in row['causa'] if c][:2]
            if causas:
                print(f"Causas: {', '.join(causas)}")
        print()
else:
    print("No se obtuvieron datos de Wikidata")
    wikidata_grouped = pd.DataFrame()

Procesando datos de Wikidata...
Total de enfermedades únicas: 216
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Abetalipoproteinemia

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Acromatopsia
Síntomas: nistagmo, fotofobia, ambliopía

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Adams-Oliver syndrome

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Adermatoglifia

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Adrenoleucodistrofia
Tratamientos: Aceite de Lorenzo
 Medicamentos: Aceite de Lorenzo



In [22]:
def prepare_neo4j_data():
  """
  Preparar datos estructurados para importar en Neo4j
  Combina información de DBpedia y Wikidata sin URLs
  """
  
  # Diccionarios para almacenar entidades únicas
  diseases = {}
  symptoms = {}
  treatments = {}
  causes = {}
  medications = {}
  relationships = []
  
  print("Procesando datos de DBpedia EN...")
  # Procesar DBpedia EN
  for idx, row in diseases_df.iterrows():
    disease_name = row['nombre'].strip()
    
    # Agregar enfermedad
    if disease_name and disease_name not in diseases:
      diseases[disease_name] = {
        'name': disease_name,
        'description': row['descripcion'].strip() if row['descripcion'] else '',
        'source': 'DBpedia EN'
      }
    
    # Procesar síntomas
    if row['sintomas']:
      for symptom_uri in row['sintomas']:
        if symptom_uri:
          symptom_name = symptom_uri.split('/')[-1].replace('_', ' ')
          if symptom_name and symptom_name not in symptoms:
            symptoms[symptom_name] = {'name': symptom_name}
          
          if symptom_name:
            relationships.append({
              'type': 'HAS_SYMPTOM',
              'from': disease_name,
              'to': symptom_name,
              'from_type': 'Disease',
              'to_type': 'Symptom'
            })
    
    # Procesar tratamientos
    if row['tratamientos']:
      for treatment_uri in row['tratamientos']:
        if treatment_uri:
          treatment_name = treatment_uri.split('/')[-1].replace('_', ' ')
          if treatment_name and treatment_name not in treatments:
            treatments[treatment_name] = {'name': treatment_name}
          
          if treatment_name:
            relationships.append({
              'type': 'TREATED_WITH',
              'from': disease_name,
              'to': treatment_name,
              'from_type': 'Disease',
              'to_type': 'Treatment'
            })
    
    # Procesar causas
    if row['causas']:
      for cause_item in row['causas']:
        if cause_item:
          # Limpiar causa (puede ser texto o URI)
          if 'http' in cause_item:
            cause_name = cause_item.split('/')[-1].replace('_', ' ')
          else:
            cause_name = cause_item.strip()
          
          if cause_name and cause_name not in causes:
            causes[cause_name] = {'name': cause_name}
          
          if cause_name:
            relationships.append({
              'type': 'CAUSED_BY',
              'from': disease_name,
              'to': cause_name,
              'from_type': 'Disease',
              'to_type': 'Cause'
            })
  
  print(f"DBpedia EN: {len(diseases)} enfermedades procesadas")
  
  print("Procesando datos de DBpedia ES...")
  # Procesar DBpedia ES (descripciones en español)
  for idx, row in diseases_es_df.iterrows():
    disease_name = row['nombre'].strip()
    description = row['descripcion'].strip()
    
    if disease_name:
      if disease_name in diseases:
        # Actualizar con descripción en español si es más completa
        if len(description) > len(diseases[disease_name].get('description', '')):
          diseases[disease_name]['description'] = description
          diseases[disease_name]['source'] = 'DBpedia ES'
      else:
        diseases[disease_name] = {
          'name': disease_name,
          'description': description,
          'source': 'DBpedia ES'
        }
  
  print(f"DBpedia ES: {len(diseases)} enfermedades totales")
  
  print("Procesando datos de Wikidata...")
  # Procesar Wikidata
  for idx, row in wikidata_grouped.iterrows():
    disease_name = row['enfermedad'].strip()
    
    # Agregar enfermedad
    if disease_name and disease_name not in diseases:
      diseases[disease_name] = {
        'name': disease_name,
        'description': '',
        'source': 'Wikidata'
      }
    
    # Procesar síntomas
    if row['sintoma']:
      for symptom_name in row['sintoma']:
        if symptom_name:
          symptom_name = symptom_name.strip()
          if symptom_name not in symptoms:
            symptoms[symptom_name] = {'name': symptom_name}
          
          relationships.append({
            'type': 'HAS_SYMPTOM',
            'from': disease_name,
            'to': symptom_name,
            'from_type': 'Disease',
            'to_type': 'Symptom'
          })
    
    # Procesar tratamientos
    if row['tratamiento']:
      for treatment_name in row['tratamiento']:
        if treatment_name:
          treatment_name = treatment_name.strip()
          if treatment_name not in treatments:
            treatments[treatment_name] = {'name': treatment_name}
          
          relationships.append({
            'type': 'TREATED_WITH',
            'from': disease_name,
            'to': treatment_name,
            'from_type': 'Disease',
            'to_type': 'Treatment'
          })
    
    # Procesar medicamentos
    if row['medicamento']:
      for medication_name in row['medicamento']:
        if medication_name:
          medication_name = medication_name.strip()
          if medication_name not in medications:
            medications[medication_name] = {'name': medication_name}
          
          relationships.append({
            'type': 'USES_MEDICATION',
            'from': disease_name,
            'to': medication_name,
            'from_type': 'Disease',
            'to_type': 'Medication'
          })
    
    # Procesar causas
    if row['causa']:
      for cause_name in row['causa']:
        if cause_name:
          cause_name = cause_name.strip()
          if cause_name not in causes:
            causes[cause_name] = {'name': cause_name}
          
          relationships.append({
            'type': 'CAUSED_BY',
            'from': disease_name,
            'to': cause_name,
            'from_type': 'Disease',
            'to_type': 'Cause'
          })
  
  print(f"Wikidata procesado")
  
  # Convertir diccionarios a listas
  neo4j_graph = {
    'nodes': {
      'diseases': [{'id': name, **data} for name, data in diseases.items()],
      'symptoms': [{'id': name, **data} for name, data in symptoms.items()],
      'treatments': [{'id': name, **data} for name, data in treatments.items()],
      'causes': [{'id': name, **data} for name, data in causes.items()],
      'medications': [{'id': name, **data} for name, data in medications.items()]
    },
    'relationships': relationships
  }
  
  # Estadísticas
  print("\n" + "="*60)
  print("RESUMEN DEL GRAFO DE CONOCIMIENTO")
  print("="*60)
  print(f"Enfermedades: {len(diseases)}")
  print(f"Sintomas: {len(symptoms)}")
  print(f"Tratamientos: {len(treatments)}")
  print(f"Medicamentos: {len(medications)}")
  print(f"Causas: {len(causes)}")
  print(f"Relaciones: {len(relationships)}")
  print("="*60)
  
  return neo4j_graph

# Preparar datos
neo4j_graph = prepare_neo4j_data()

Procesando datos de DBpedia EN...
DBpedia EN: 299 enfermedades procesadas
Procesando datos de DBpedia ES...
DBpedia ES: 495 enfermedades totales
Procesando datos de Wikidata...
Wikidata procesado

RESUMEN DEL GRAFO DE CONOCIMIENTO
Enfermedades: 705
Sintomas: 160
Tratamientos: 184
Medicamentos: 4
Causas: 87
Relaciones: 499


In [23]:
from datetime import datetime
import pickle

def save_knowledge_graph(neo4j_graph: Dict, base_filename: str = "medical_knowledge_graph") -> Dict[str, str]:
  """
  Guardar el grafo de conocimiento en múltiples formatos
  
  Args:
    neo4j_graph: Diccionario con nodos y relaciones del grafo
    base_filename: Nombre base para los archivos
    
  Returns:
    Diccionario con las rutas de los archivos guardados
  """
  
  # Generar timestamp para versionar archivos
  timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
  
  saved_files = {}
  
  # 1. Guardar en JSON (legible, para importar en Neo4j)
  json_filename = f"{base_filename}_{timestamp}.json"
  with open(json_filename, 'w', encoding='utf-8') as f:
    json.dump(neo4j_graph, f, ensure_ascii=False, indent=2)
  saved_files['json'] = json_filename
  print(f"✓ Guardado en JSON: {json_filename}")
  
  # 2. Guardar en Pickle (rápido, para Python)
  pickle_filename = f"{base_filename}_{timestamp}.pkl"
  with open(pickle_filename, 'wb') as f:
    pickle.dump(neo4j_graph, f)
  saved_files['pickle'] = pickle_filename
  print(f"✓ Guardado en Pickle: {pickle_filename}")
  
  # 3. Guardar resumen en texto
  summary_filename = f"medical_kg_summary_{timestamp}.txt"
  with open(summary_filename, 'w', encoding='utf-8') as f:
    f.write("=" * 70 + "\n")
    f.write("RESUMEN DEL GRAFO DE CONOCIMIENTO MÉDICO\n")
    f.write("=" * 70 + "\n\n")
    
    f.write(f"Fecha de generación: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    
    # Estadísticas de nodos
    f.write("NODOS:\n")
    f.write("-" * 70 + "\n")
    for node_type, nodes in neo4j_graph['nodes'].items():
      f.write(f"  {node_type.capitalize()}: {len(nodes)}\n")
    
    f.write(f"\nRELACIONES: {len(neo4j_graph['relationships'])}\n")
    f.write("-" * 70 + "\n")
    
    # Contar tipos de relaciones
    rel_types = {}
    for rel in neo4j_graph['relationships']:
      rel_type = rel['type']
      rel_types[rel_type] = rel_types.get(rel_type, 0) + 1
    
    for rel_type, count in sorted(rel_types.items(), key=lambda x: x[1], reverse=True):
      f.write(f"  {rel_type}: {count}\n")
    
    # Muestra de enfermedades con más información
    f.write("\n" + "=" * 70 + "\n")
    f.write("MUESTRA DE ENFERMEDADES CON MÁS RELACIONES\n")
    f.write("=" * 70 + "\n\n")
    
    # Contar relaciones por enfermedad
    disease_rels = {}
    for rel in neo4j_graph['relationships']:
      if rel['from_type'] == 'Disease':
        disease = rel['from']
        disease_rels[disease] = disease_rels.get(disease, 0) + 1
    
    # Top 10 enfermedades
    top_diseases = sorted(disease_rels.items(), key=lambda x: x[1], reverse=True)[:10]
    
    for disease_name, rel_count in top_diseases:
      # Buscar información de la enfermedad
      disease_info = next((d for d in neo4j_graph['nodes']['diseases'] if d['id'] == disease_name), None)
      
      if disease_info:
        f.write(f"\n{disease_name}\n")
        f.write(f"  Relaciones: {rel_count}\n")
        if disease_info.get('description'):
          desc = disease_info['description'][:200] + "..." if len(disease_info['description']) > 200 else disease_info['description']
          f.write(f"  Descripción: {desc}\n")
        f.write(f"  Fuente: {disease_info.get('source', 'N/A')}\n")
    
    # Fuentes de datos
    f.write("\n" + "=" * 70 + "\n")
    f.write("FUENTES DE DATOS\n")
    f.write("=" * 70 + "\n")
    sources = set(d.get('source', 'N/A') for d in neo4j_graph['nodes']['diseases'])
    for source in sources:
      count = sum(1 for d in neo4j_graph['nodes']['diseases'] if d.get('source') == source)
      f.write(f"  {source}: {count} enfermedades\n")
  
  saved_files['summary'] = summary_filename
  print(f"✓ Guardado resumen en: {summary_filename}")
  
  # 4. Crear archivo CSV de nodos (para análisis en pandas)
  nodes_csv_filename = f"medical_kg_nodes_{timestamp}.csv"
  all_nodes = []
  for node_type, nodes in neo4j_graph['nodes'].items():
    for node in nodes:
      all_nodes.append({
        'node_type': node_type,
        'id': node['id'],
        'name': node.get('name', ''),
        'description': node.get('description', ''),
        'source': node.get('source', '')
      })
  
  nodes_df = pd.DataFrame(all_nodes)
  nodes_df.to_csv(nodes_csv_filename, index=False, encoding='utf-8')
  saved_files['nodes_csv'] = nodes_csv_filename
  print(f"✓ Guardados nodos en CSV: {nodes_csv_filename}")
  
  # 5. Crear archivo CSV de relaciones
  rels_csv_filename = f"medical_kg_relationships_{timestamp}.csv"
  rels_df = pd.DataFrame(neo4j_graph['relationships'])
  rels_df.to_csv(rels_csv_filename, index=False, encoding='utf-8')
  saved_files['relationships_csv'] = rels_csv_filename
  print(f"✓ Guardadas relaciones en CSV: {rels_csv_filename}")
  
  print("\n" + "=" * 70)
  print("ARCHIVOS GUARDADOS EXITOSAMENTE")
  print("=" * 70)
  
  return saved_files

# Guardar resultados
saved_files = save_knowledge_graph(neo4j_graph)

✓ Guardado en JSON: medical_knowledge_graph_20251120_211246.json
✓ Guardado en Pickle: medical_knowledge_graph_20251120_211246.pkl
✓ Guardado resumen en: medical_kg_summary_20251120_211246.txt
✓ Guardados nodos en CSV: medical_kg_nodes_20251120_211246.csv
✓ Guardadas relaciones en CSV: medical_kg_relationships_20251120_211246.csv

ARCHIVOS GUARDADOS EXITOSAMENTE
